https://colab.research.google.com/github/Adapter-Hub/adapters/blob/main/notebooks/01_Adapter_Training.ipynb#scrollTo=hnct-N6dLvOd

In [1]:
!pip install -qq adapters datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 59.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("hrithikpiyush/acl-arc")
dataset = dataset.rename_column(original_column_name="intent", new_column_name="labels")

train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1688 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/114 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/139 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/cs_roberta_base")

def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

tokenized_train_data = train_data.map(tokenize_function, batched=True, batch_size=len(train_data))
tokenized_val_data = val_data.map(tokenize_function, batched=True, batch_size=len(val_data))
tokenized_test_data = test_data.map(tokenize_function, batched=True, batch_size=len(test_data))

tokenizer_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Map:   0%|          | 0/1688 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

In [4]:
tokenized_train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [5]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

num_labels = len(set(train_data['labels']))

config = RobertaConfig.from_pretrained(
    "allenai/cs_roberta_base",
    num_labels=num_labels,
)
model = AutoAdapterModel.from_pretrained(
    "allenai/cs_roberta_base",
    config=config,
)

pytorch_model.bin:   0%|          | 0.00/656M [00:00<?, ?B/s]

Basically, train_adapter() method performs two functions majorly:
*   It freezes all the weights of the pre-trained model such that only the adapter weights are updated during the training.
*   It also activates the adapter and prediction head to use both in every forward pass.

In [6]:
# Add a new adapter
model.add_adapter("acl-arc", config="par_bn")
# Alternatively, e.g.:
# model.add_adapter("rotten_tomatoes", config="lora")

# Add a matching classification head
model.add_classification_head(
    "acl-arc",
    num_labels=num_labels,
    id2label={
        0: "label1_0",
        1: "label1_1",
        2: "label1_2",
        3: "label1_3",
        4: "label1_4",
        5: "label1_5",
    }
)

# Activate the adapter
model.train_adapter("acl-arc")

In [7]:
import numpy as np
import evaluate

# https://huggingface.co/spaces/evaluate-metric/f1/blob/main/f1.py
# Example 5-A multi-label example
metric = evaluate.load("f1", "multilabel")

In [8]:
# https://medium.com/@rakeshrajpurohit/customized-evaluation-metrics-with-hugging-face-trainer-3ff00d936f99

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision_macro = precision_score(labels, preds, average='macro')
    precision_weighted = precision_score(labels, preds, average='weighted')
    recall_macro = recall_score(labels, preds, average='macro')
    recall_weighted = recall_score(labels, preds, average='weighted')
    f1_macro = f1_score(labels, preds, average='macro')
    f1_weighted = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_weighted': precision_weighted,
        'recall_macro': recall_macro,
        'recall_weighted': recall_weighted,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=5,  # <- change from 6 to 5
    per_device_train_batch_size=32,  # <- default is 8 idk what is better
    per_device_eval_batch_size=32,  # <- default is 8
    auto_find_batch_size=True,  # automates lowering the batch size to resolve out-of-memory errors
    logging_steps=100,
    output_dir="./adapter_cs_roberta_base_par_bn_acl_arc_1",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy="epoch"
)

In [10]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Precision Weighted,Recall Macro,Recall Weighted,F1 Macro,F1 Weighted
1,No log,1.231258,0.517544,0.086257,0.267852,0.166667,0.517544,0.113680,0.353007
2,1.273100,0.911879,0.692982,0.384421,0.668476,0.335620,0.692982,0.297637,0.606339
3,1.273100,0.875297,0.728070,0.344231,0.652260,0.387813,0.728070,0.358349,0.681017
4,0.941200,0.744740,0.736842,0.587361,0.705074,0.535782,0.736842,0.550629,0.717564
5,0.941200,0.741397,0.736842,0.471752,0.688256,0.507163,0.736842,0.484867,0.707397


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=265, training_loss=1.0195748095242483, metrics={'train_runtime': 350.4241, 'train_samples_per_second': 24.085, 'train_steps_per_second': 0.756, 'total_flos': 2419921380556800.0, 'train_loss': 1.0195748095242483, 'epoch': 5.0})

In [12]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.7413973212242126,
 'eval_accuracy': 0.7368421052631579,
 'eval_precision_macro': 0.47175166970248944,
 'eval_precision_weighted': 0.6882561595244943,
 'eval_recall_macro': 0.5071630347054076,
 'eval_recall_weighted': 0.7368421052631579,
 'eval_f1_macro': 0.48486676721970845,
 'eval_f1_weighted': 0.7073966023656427,
 'eval_runtime': 2.1199,
 'eval_samples_per_second': 53.777,
 'eval_steps_per_second': 1.887,
 'epoch': 5.0}

In [13]:
trainer.predict(tokenized_test_data)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[ 5.80949903e-01, -1.67563343e+00,  2.42937326e+00,
        -9.56665456e-01, -4.07156080e-01, -8.84354532e-01],
       [ 2.68875313e+00, -1.96307623e+00,  1.82661390e+00,
        -1.94076359e+00,  1.43880695e-01, -1.17072761e+00],
       [ 3.86655641e+00, -1.18546820e+00,  3.62072170e-01,
        -2.30156088e+00,  2.17274174e-01, -1.22074568e+00],
       [ 2.64445591e+00, -1.68981874e+00,  1.50071120e+00,
        -1.81503546e+00, -6.06879219e-02, -1.17414820e+00],
       [-2.73130536e-01, -3.56890559e-01, -4.61255610e-01,
         1.96714818e-01, -3.53633761e-01,  7.28782833e-01],
       [ 3.59197497e+00, -7.46937990e-01,  4.10891652e-01,
        -2.43265820e+00,  2.33795688e-01, -1.35880780e+00],
       [ 2.33590150e+00, -1.02545179e-01,  6.13013506e-01,
        -1.78701508e+00, -3.70072484e-01, -1.40571010e+00],
       [ 3.46652031e+00, -6.88272774e-01,  1.76441222e-02,
        -2.00882840e+00, -1.70562521e-01, -1.10934746e+00],
       [ 3.69735646